In [1]:
%cd /content/drive/MyDrive/Aerial_Scene_Recognition

/content/drive/MyDrive/Aerial_Scene_Recognition


In [2]:
import logging
logging.basicConfig(level=logging.INFO, force=True)

In [3]:
!pip install -r requirements.txt --quiet

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 37.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.6/151.6 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 890.1/890.1 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.3/24.3 MB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 101.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.0/510.0 kB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 104.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━

In [4]:
import os
current_directory = os.getcwd()

In [5]:
import torch
from models import imagebind_model
from models.imagebind_model import ModalityType, load_module
from models import lora as LoRA

device = "cpu"

In [6]:
model = imagebind_model.imagebind_huge(pretrained=True)

In [7]:
lora = True
linear_probing = False
load_head_post_proc_finetuned = True

In [8]:
if lora:
    model.modality_trunks.update(
        LoRA.apply_lora_modality_trunks(model.modality_trunks, rank=4,
                                        modality_names=[ModalityType.VISION, ModalityType.AUDIO]))

    LoRA.load_lora_modality_trunks(model.modality_trunks,
                                   checkpoint_dir=".checkpoints/lora/", postfix="_last")

    if load_head_post_proc_finetuned:
        load_module(model.modality_postprocessors, module_name="postprocessors",
                    checkpoint_dir=".checkpoints/lora/", postfix="_last")
        load_module(model.modality_heads, module_name="heads",
                    checkpoint_dir=".checkpoints/lora/", postfix="_last")
elif linear_probing:
    load_module(model.modality_heads, module_name="heads",
                checkpoint_dir="./.checkpoints/lora/", postfix="_last")

INFO:root:Loaded LoRA parameters for modality vision from .checkpoints/lora/.
INFO:root:Loaded LoRA parameters for modality audio from .checkpoints/lora/.
INFO:root:Loaded parameters for module postprocessors from .checkpoints/lora/.
INFO:root:Loaded parameters for module heads from .checkpoints/lora/.


In [9]:
model.eval()
model.to(device)

ImageBindModel(
  (modality_preprocessors): ModuleDict(
    (vision): RGBDTPreprocessor(
      (cls_token): tensor((1, 1, 1280), requires_grad=True)
      
      (rgbt_stem): PatchEmbedGeneric(
        (proj): Sequential(
          (0): PadIm2Video()
          (1): Conv3d(3, 1280, kernel_size=(2, 14, 14), stride=(2, 14, 14), bias=False)
        )
      )
      (pos_embedding_helper): SpatioTemporalPosEmbeddingHelper(
        (pos_embed): tensor((1, 257, 1280), requires_grad=True)
        
      )
    )
    (text): TextPreprocessor(
      (pos_embed): tensor((1, 77, 1024), requires_grad=True)
      (mask): tensor((77, 77), requires_grad=False)
      
      (token_embedding): Embedding(49408, 1024)
    )
    (audio): AudioPreprocessor(
      (cls_token): tensor((1, 1, 768), requires_grad=True)
      
      (rgbt_stem): PatchEmbedGeneric(
        (proj): Conv2d(1, 768, kernel_size=(16, 16), stride=(10, 10), bias=False)
        (norm_layer): LayerNorm((768,), eps=1e-05, elementwise_affine=

In [10]:
loaded_inputs = torch.load('train_val_inputs.pth', map_location=torch.device('cpu'))

In [11]:
loaded_inputs['vision'].shape

torch.Size([4012, 3, 224, 224])

In [12]:
loaded_inputs['audio'].shape

torch.Size([4012, 3, 1, 128, 204])

In [13]:
import h5py
with h5py.File('ClassificationAfterFinetune/train_val_vision_embeddings.h5', 'w') as h5f:
  num_embeddings = loaded_inputs['vision'].shape[0]
  embedding_size = 1024
  dataset = h5f.create_dataset('ClassificationAfterFinetune/train_val_vision_embeddings', (num_embeddings, embedding_size), dtype='f')
  for i in range(num_embeddings):
        embd = model({'vision': torch.unsqueeze(loaded_inputs['vision'][i], dim=0)})
        dataset[i] = embd['vision'].detach().cpu().numpy()
        del embd

KeyboardInterrupt: 

In [ ]:
import h5py
with h5py.File('ClassificationAfterFinetune/train_val_audio_embeddings.h5', 'w') as h5f:
  num_embeddings = loaded_inputs['audio'].shape[0]
  embedding_size = 1024
  dataset = h5f.create_dataset('ClassificationAfterFinetune/train_val_audio_embeddings', (num_embeddings, embedding_size), dtype='f')
  for i in range(num_embeddings):
        embd = model({'audio': torch.unsqueeze(loaded_inputs['audio'][i], dim=0)})
        dataset[i] = embd['audio'].detach().cpu().numpy()
        del embd